<a href="https://colab.research.google.com/github/BenyaminZojaji/Deep_Learning/blob/main/PyTorch%20Mnist/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torchvision
from tqdm import tqdm

In [11]:
def calc_acc(preds, labels):
  _, preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [2]:
class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(28*28, 128)
    self.fc2 = torch.nn.Linear(128, 10)

  def forward(self, x):
    # input shape: 32 * 28 * 28 * 1 <-> batch-size * height * width * channel
    x = x.reshape((x.shape[0], 784))
    # output shape: 32 * 784

    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2, train=True)
    x = self.fc2(x)
    x = torch.softmax(x, dim=1)
    return x

In [ ]:
device = torch.device('cuda')
# device = torch.device('cpu')

model = MyModel()
model = model.to(device)
model.train(True)

In [4]:
# Hyper Parameters
batch_size = 64
epochs = 10
lr = 0.001

In [5]:
# Data Preparing

transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0), (1))
])

In [6]:
!mkdir dataset

In [ ]:
dataset = torchvision.datasets.MNIST('./dataset', train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [10]:
#
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [14]:
# train

for epoch in range(epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in tqdm(train_data_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()

    # 1- forwarding
    preds = model(images)

    # 2- backwarding
    loss = loss_function(preds, labels)
    loss.backward()

    # 3- Update
    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)

  total_loss = train_loss / len(train_data_loader)
  total_acc = train_acc / len(train_data_loader)

  print(f'Epoch: {epoch+1}, Loss: {total_loss}, Acc: {total_acc}')


100%|██████████| 938/938 [00:14<00:00, 66.91it/s]


Epoch: 1, Loss: 1.6186983585357666, Acc: 0.8718516791044776


100%|██████████| 938/938 [00:13<00:00, 67.79it/s]


Epoch: 2, Loss: 1.5392043590545654, Acc: 0.9298540778251599


100%|██████████| 938/938 [00:13<00:00, 67.25it/s]


...

Epoch: 8, Loss: 1.4958008527755737, Acc: 0.968566764392324


100%|██████████| 938/938 [00:13<00:00, 67.28it/s]


Epoch: 9, Loss: 1.4923021793365479, Acc: 0.9717484008528784


100%|██████████| 938/938 [00:13<00:00, 67.09it/s]

Epoch: 10, Loss: 1.4915443658828735, Acc: 0.9722314765458422


In [ ]:
# save
torch.save(model.state_dict(), 'mnist.pth')

In [19]:
# inference

import cv2
import numpy as np

#model.train(False)
model.eval()

# oreoricess
img = cv2.imread('3.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
tensor = transform(img).unsqueeze(0).to(device)

# process
preds = model(tensor)

# postprocess
# _, output=torch.max(preds, 1)
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
print(output)

3
